# Importing Libraries

In [109]:
import os
import sys
import pickle
import datetime
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_selection import SelectPercentile, f_classif, chi2


import tensorflow as tf
from tensorflow.keras import Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization

# Pathing & Static variable

In [110]:
CURRENT_DIR = os.getcwd()
PROJECT_DIR = os.path.dirname(CURRENT_DIR)
MODELS_PATH = os.path.join(PROJECT_DIR, 'models')
ENCODER_PATH = os.path.join(MODELS_PATH, 'encoders')
LOGS_PATH = os.path.join(MODELS_PATH, 'logs')
LOG_CURRENT = os.path.join(LOGS_PATH, datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
TEST_DATA_PATH = os.path.join(PROJECT_DIR, 
                              'data', 'raw', 'carInsurance_train.csv')

CATEG_PATH = os.path.join(PROJECT_DIR, 'references', 'categorical_columns.txt')
CONTI_PATH = os.path.join(PROJECT_DIR, 'references', 'continous_columns.txt')

PROJECT_NAME = '2.1-ie-Linear-SVC-model'
MODEL_NAME = 'LinearSVC-v1.0'

In [111]:
# adding system path
sys.path.insert(0, PROJECT_DIR)

In [112]:
# import internal function
from src.data import process_pipeline, encoder_pipeline, feature_selection_pipeline

# Helper Function

In [113]:
def get_content(txt_file):
    contents = []
    with open(txt_file) as file:
        for line in file:
            contents.append(line.strip())
            
    return contents

# Function to save a trained model
def save_model(model, model_name, folderPath):
    filename = os.path.join(folderPath, f"{model_name}_model.pkl")
    with open(filename, 'wb') as file:
        pickle.dump(model, file)

# Function to load a saved model
def load_model(model_name):
    filename = f"{model_name}_model.pkl"
    if os.path.exists(filename):
        with open(filename, 'rb') as file:
            return pickle.load(file)
    else:
        return None

# Function to evaluate a model
def evaluate_model(model, model_name, X_test, y_test):
    y_pred = model.predict(X_test)

    # DL model return float type
    if y_pred.dtype != 'int':
        y_pred = y_pred.astype('int')
        
    accuracy = accuracy_score(y_test, y_pred)
    print(f"{model_name} - Test Accuracy: {accuracy:.4f}")
    return accuracy

# call backs
def callbacks(LOGS_PATH, patience):
    tensorboard_callbacks = TensorBoard(log_dir=LOGS_PATH, histogram_freq=1)
    early_callback = EarlyStopping(monitor='accuracy', patience=patience)
    return [early_callback, tensorboard_callbacks]

# Data Loading

In [114]:
df = pd.read_csv(TEST_DATA_PATH)
df = process_pipeline.process_data(df)
df.head()

,Id,Age,Job,Marital,Education,Default,Balance,HHInsurance,CarLoan,Communication,...,PrevAttempts,Outcome,CallStart,CallEnd,CarInsurance,HasCommuncation,CallDuration,CallCategory,DaysPassed_Simplify,Outcome_Simplify
0,1,32,management,single,tertiary,0,1218,1,0,telephone,...,0,failure,49520,49590,0,1,70.0,Afternoon,0,0
1,2,32,blue-collar,married,primary,0,1156,1,0,No Communication,...,0,failure,53343,53528,0,0,185.0,Afternoon,0,0
2,3,29,management,single,tertiary,0,637,1,0,cellular,...,1,failure,59424,59764,1,1,340.0,Afternoon,0,0
3,4,25,student,single,primary,0,373,1,0,cellular,...,0,failure,43603,44422,1,1,819.0,Afternoon,0,0
4,5,30,management,married,tertiary,0,2694,0,0,cellular,...,0,failure,52544,52736,0,1,192.0,Afternoon,0,0


In [115]:
# Get list of categorical & continous variable
categ = get_content(CATEG_PATH)
conti = get_content(CONTI_PATH)

In [116]:
# making sure there's no target
categ

['Age',
 'Job',
 'Marital',
 'Education',
 'Default',
 'HHInsurance',
 'CarLoan',
 'Communication',
 'HasCommuncation',
 'LastContactDay',
 'LastContactMonth',
 'NoOfContacts',
 'PrevAttempts',
 'Outcome',
 'Outcome_Simplify',
 'CallCategory']

In [117]:
# label
X = df.drop(columns=['CarInsurance'])

#target
y = df['CarInsurance']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [118]:
df

,Id,Age,Job,Marital,Education,Default,Balance,HHInsurance,CarLoan,Communication,...,PrevAttempts,Outcome,CallStart,CallEnd,CarInsurance,HasCommuncation,CallDuration,CallCategory,DaysPassed_Simplify,Outcome_Simplify
0,1,32,management,single,tertiary,0,1218,1,0,telephone,...,0,failure,49520,49590,0,1,70.0,Afternoon,0,0
1,2,32,blue-collar,married,primary,0,1156,1,0,No Communication,...,0,failure,53343,53528,0,0,185.0,Afternoon,0,0
2,3,29,management,single,tertiary,0,637,1,0,cellular,...,1,failure,59424,59764,1,1,340.0,Afternoon,0,0
3,4,25,student,single,primary,0,373,1,0,cellular,...,0,failure,43603,44422,1,1,819.0,Afternoon,0,0
4,5,30,management,married,tertiary,0,2694,0,0,cellular,...,0,failure,52544,52736,0,1,192.0,Afternoon,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,3996,28,technician,single,tertiary,0,0,1,0,cellular,...,2,failure,63988,64257,1,1,269.0,Afternoon,0,0
3996,3997,49,admin.,divorced,secondary,0,124,1,1,cellular,...,0,failure,53356,53481,0,1,125.0,Afternoon,0,0
3997,3998,27,admin.,single,secondary,0,-400,0,1,cellular,...,0,failure,44343,44633,0,1,290.0,Afternoon,0,0
3998,3999,36,entrepreneur,single,tertiary,0,658,1,0,cellular,...,3,failure,41255,41354,0,1,99.0,Morning,0,0


# Data Preprocessing

In [119]:
# Numeric feature processing
numeric_transformer = make_pipeline(
    # scaling
    StandardScaler()
)

# Categorical feature processing
ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
categorical_transformer = make_pipeline(
    ordinal_encoder,
    StandardScaler()
)

# combine pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, conti),
        ("cat", categorical_transformer, categ)
    ]
)

In [120]:
# transform to get the shape
scaler_process = make_pipeline(preprocessor)
x_train = scaler_process.fit_transform(X_train)
x_test = scaler_process.transform(X_test)

# create a input and output shape
input_shape = np.shape(x_train)[1:]
model_output = 1 # len(np.unique(y_train, axis=0)) # alternative len(df.target.unique)

In [141]:
# manual testing
model = Sequential()
model.add(Input(shape=input_shape))

model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))

model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))

model.add(Dense(model_output, activation='sigmoid'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(
    x_train, y_train,
    epochs=50,
    batch_size=32,
    validation_data=(x_test, y_test),
    callbacks=[EarlyStopping(monitor='val_accuracy', patience=25, restore_best_weights=True)]
)

Epoch 1/50
100/100 [==============================] - 1s 8ms/step - loss: 0.0000e+00 - accuracy: 0.7750 - val_loss: 0.0000e+00 - val_accuracy: 0.8112
Epoch 2/50
100/100 [==============================] - 1s 7ms/step - loss: 0.0000e+00 - accuracy: 0.8047 - val_loss: 0.0000e+00 - val_accuracy: 0.8188
Epoch 3/50
100/100 [==============================] - 1s 6ms/step - loss: 0.0000e+00 - accuracy: 0.8184 - val_loss: 0.0000e+00 - val_accuracy: 0.8200
Epoch 4/50
100/100 [==============================] - 1s 7ms/step - loss: 0.0000e+00 - accuracy: 0.8209 - val_loss: 0.0000e+00 - val_accuracy: 0.8275
Epoch 5/50
100/100 [==============================] - 1s 6ms/step - loss: 0.0000e+00 - accuracy: 0.8194 - val_loss: 0.0000e+00 - val_accuracy: 0.8150
Epoch 6/50
100/100 [==============================] - 1s 6ms/step - loss: 0.0000e+00 - accuracy: 0.8197 - val_loss: 0.0000e+00 - val_accuracy: 0.8138
Epoch 7/50
100/100 [==============================] - 1s 6ms/step - loss: 0.0000e+00 - accuracy: 0.8

In [142]:
evaluate_model(model, 'manual_keras', x_test, y_test)

25/25 [==============================] - 0s 2ms/step
manual_keras - Test Accuracy: 0.8237


0.82375

In [131]:
def create_model(input_shape, model_output):
    model = Sequential()
    # input layer
    model.add(Input(shape=input_shape))

    
    # layer 1
    model.add(Dense(256, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    # layer 2
    model.add(Dense(512, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))

    # output
    model.add(Dense(model_output, activation='sigmoid'))
    model.compile(optimizer='adam', 
                  loss='binary_crossentropy', 
                  metrics=['accuracy']
                 )
    return model

In [132]:
# callbacks
early_callback, tensorboard_callbacks = callbacks(LOGS_PATH, patience=100)

In [133]:
clf = KerasClassifier(
    build_fn=lambda : create_model(input_shape, model_output),
    epochs=10,
    batch_size=32,
    callbacks=[early_callback, tensorboard_callbacks],
    verbose=0
)

C:\Users\isaac\AppData\Local\Temp\ipykernel_1524\836831083.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  clf = KerasClassifier(


In [134]:
clf.build_fn().summary()

Model: "sequential_37"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_130 (Dense)           (None, 256)               5632      
                                                                 
 batch_normalization_70 (Bat  (None, 256)              1024      
 chNormalization)                                                
                                                                 
 dropout_70 (Dropout)        (None, 256)               0         
                                                                 
 dense_131 (Dense)           (None, 512)               131584    
                                                                 
 batch_normalization_71 (Bat  (None, 512)              2048      
 chNormalization)                                                
                                                                 
 dropout_71 (Dropout)        (None, 512)             

In [135]:
pipeline = make_pipeline(
    preprocessor,
    StandardScaler(),
    clf
)

In [136]:
# model training
pipeline.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('standardscaler',
                                                                   StandardScaler())]),
                                                  ['Balance', 'DaysPassed',
                                                   'CallStart', 'CallEnd',
                                                   'CallDuration']),
                                                 ('cat',
                                                  Pipeline(steps=[('ordinalencoder',
                                                                   OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                                  unknown_value=-1)),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  ['Age', 'Job', 'Marital',
                                                   'Education', 'Default',
                                                   'HHInsurance', 'CarLoan',
                                                   'Communication',
                                                   'HasCommuncation',
                                                   'LastContactDay',
                                                   'LastContactMonth',
                                                   'NoOfContacts',
                                                   'PrevAttempts', 'Outcome',
                                                   'Outcome_Simplify',
                                                   'CallCategory'])])),
                ('standardscaler', StandardScaler()),
                ('kerasclassifier',
                 <keras.wrappers.scikit_learn.KerasClassifier object at 0x000002032CAA5640>)])

In [137]:
evaluate_model(pipeline, 'keras', X_test, y_test)

25/25 [==============================] - 0s 3ms/step
keras - Test Accuracy: 0.8237


0.82375

In [138]:
# try:
#     scores = cross_val_score(pipeline, X_train, y_train, cv=5, scoring='accuracy')
#     print(f"Sequential DL: Accuracy - {scores.mean():.4f} (± {scores.std():.4f})")
    
# except Exception as e:
#     print(f"Sequential DL: Fit failed with error: {e}")